In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt

In [6]:
# CIFAR_10 is a set of 60K colour (RGB, 3 channels) images of 32x32 pixels in size.
# There's 10 classes, each with 6000 images.
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32
BATCH_SIZE = 128
NB_EPOCH = 20
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = Adam()

In [ ]:
# Load CIFAR10 from Keras.
# ? Because it's a builtin Keras dataset, it's directly loaded as test and train sets.
# ? It's not great practice as you can't do that with your pneumonia set.
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# One-hot encoding, done automatically using to_categorical.
Y_train = to_categorical(y_train, NB_CLASSES)
Y_test = to_categorical(y_test, NB_CLASSES)

# Float typing and data normalisation
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


The second answer in [this StackOverflow question](https://stats.stackexchange.com/questions/296679/what-does-kernel-size-mean)
has a useful GIF explaining how the convolution filter works.

In [ ]:
model = Sequential()

# ? Convoluitional layer dividng 3x3 pixel groups into features.
# ? "Same" padding as in Lab 6 to output the same image size as input.
# ? Because it's the first layer, we can specify input_shape.
# ! However, Keras doesn't like this, and instead recommends adding an actual Input() layer.
model.add(Conv2D(32, kernel_size = (3, 3), padding = "same",
                 input_shape = (IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
# * 32 because the images are 32x32.

model.add(Activation("relu"))

# ? Pooling layer grabbing 2x2 pixel clusters.
# ! This week, strides weren't specified. I'm guessing it'll default to == pool size?
model.add(MaxPooling2D(pool_size=(2, 2)))

# ? To reduce overfitting, 25% chance of dropping inputs here.
# ? Makes the model less reliant on certain neurons (i.e. specific parts of the image)
model.add(Dropout(0.25))

# ? Flatten the pooled features for interpretation by a dense layer.
model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# ? Final dense layer must always be the amount of classes. The highest weighted node here is the predicted
# ? class. For example, node 1 may be 0.0002, but node 3 is 0.43, so we go for node 3 as prediction.
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)                    │ (None, 32, 32, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │       4,194,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           5,130 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_5 (Activation)            │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,200,842 (16.02 MB)

 Trainable params: 4,200,842 (16.02 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Compile & train the model.
model.compile(loss = 'categorical_crossentropy', optimizer = OPTIM,
              metrics = ['accuracy'])

model.fit(X_train, Y_train, batch_size = BATCH_SIZE,
          epochs = NB_EPOCH, validation_split = VALIDATION_SPLIT,
          verbose = VERBOSE)

Epoch 1/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.2773 - loss: 2.0489 - val_accuracy: 0.5098 - val_loss: 1.4005
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.4851 - loss: 1.4354 - val_accuracy: 0.5576 - val_loss: 1.2827
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5342 - loss: 1.3105 - val_accuracy: 0.5883 - val_loss: 1.1864
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5584 - loss: 1.2406 - val_accuracy: 0.6009 - val_loss: 1.1426
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5838 - loss: 1.1639 - val_accuracy: 0.6145 - val_loss: 1.1131
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6012 - loss: 1.1224 - val_accuracy: 0.6226 - val_loss: 1.0797
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6159 - loss: 1.0781 - val_accuracy: 0.6332 - val_loss: 1.0611
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6333 - loss: 1.0425 - val_accuracy: 0

In [10]:
# Evaluate the model's performance.
score = model.evaluate(X_test, Y_test, batch_size = BATCH_SIZE, verbose = VERBOSE)

# ! New content - Saving the model.
print("Test score:", score[0])
print('Test accuracy:', score[1])
# save the model
model_json = model.to_json()

with open('cifar10_architecture.json', 'w') as file:
    file.write(model_json)

# The weights learned by the model on the training set
model.save_weights('cifar10.weights.h5', overwrite=True)

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6590 - loss: 0.9703
Test score: 0.982984185218811
Test accuracy: 0.6568999886512756


In [ ]:

# ! This cell is a theoretical demo for laptop use to avoid having to train the model on its low-grade CPU.
# ! After training on Colab or desktop and getting the weights file, we can load from those weights.
model.load_weights("cifar10.weights.h5")